#Import Libraries

In [ ]:
import pandas as pd 
import numpy as np

# library for data extraction
import requests
from bs4 import BeautifulSoup

# Import regex for text cleaning
import re
import string

# library to tokenize word, sentence and to remove stopwords.
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

#stop words
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# library for syllable count

In [ ]:
pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import textstat
from textstat.textstat import textstatistics

In [ ]:
# library to expand abbrivation

In [ ]:
pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import contractions

#Load Input Files

In [ ]:
df=pd.read_csv("/content/Input.xlsx - Sheet1.csv")
df.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


#Data Extraction

In [ ]:
def fetch_info(url):

  ''' This function will extract the data from url'''

  headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
  web_page=requests.get(url,headers=headers).text
  soup=BeautifulSoup(web_page, 'lxml')               # using lxml parser to parse html files
  content=soup.find('div', class_='td-post-content') #it will extract all the information from class td-post-content
  content=str(content)                               # converting into string form
  return content

In [ ]:
df['content']=df['URL'].apply(fetch_info)
df.head()

,URL_ID,URL,content
0,37,https://insights.blackcoffer.com/ai-in-healthc...,"<div class=""td-post-content"">\n<p><strong>Intr..."
1,38,https://insights.blackcoffer.com/what-if-the-c...,"<div class=""td-post-content"">\n<p>Human minds,..."
2,39,https://insights.blackcoffer.com/what-jobs-wil...,"<div class=""td-post-content"">\n<p><strong>Intr..."
3,40,https://insights.blackcoffer.com/will-machine-...,"<div class=""td-post-content"">\n<p>“Anything th..."
4,41,https://insights.blackcoffer.com/will-ai-repla...,"<div class=""td-post-content"">\n<p>“Machine int..."


#Data Cleaning

In [ ]:

def data_cleaning(data):
  
  '''This function will clean the extracted data and it will reaturn it in readable form''' 

  #remove front and trail spaces
  data = data.strip()

  #remove html tags
  data=re.sub(r"<.*?>",'',data)

  #remove /n 
  data=data.replace('\n'," ")

  #remove URL
  data=re.sub(r"https?://\S+|www\.\S+",'',data)

  #expand abbrivation
  expanded_words = []   

  for word in data.split():
    expanded_words.append(contractions.fix(word)) # using contractions.fix to expand the shortened words

  data = ' '.join(expanded_words)

  #remove double spaces
  data = data.replace("  ",' ')

  return data

In [ ]:
#applying data_cleaning function on content column 
df['content']=df['content'].apply(data_cleaning)

In [ ]:
df.head()

,URL_ID,URL,content
0,37,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 millio...
1,38,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying ..."
2,39,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the emp...
3,40,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-than...
4,41,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention th...


#Sentiment Analysis

Positive Score, Negative Score , Polarity Score and Subjectivity Score

In [ ]:
#Import all th stop words 

stop_wards1=pd.read_csv("/content/drive/MyDrive/Internship assignment /20211030 Test Assignment/StopWords/StopWords_Auditor.txt",header=None)
stop_words2=pd.read_csv("/content/drive/MyDrive/Internship assignment /20211030 Test Assignment/StopWords/StopWords_Currencies.txt", sep='|', names=['col1','col2'], encoding='latin-1')
stop_words3=pd.read_csv("/content/drive/MyDrive/Internship assignment /20211030 Test Assignment/StopWords/StopWords_DatesandNumbers.txt",header=None)
stop_words4=pd.read_csv("/content/drive/MyDrive/Internship assignment /20211030 Test Assignment/StopWords/StopWords_Generic.txt",header=None)
stop_words5=pd.read_csv("/content/drive/MyDrive/Internship assignment /20211030 Test Assignment/StopWords/StopWords_GenericLong.txt",header=None)
stop_words6=pd.read_csv("/content/drive/MyDrive/Internship assignment /20211030 Test Assignment/StopWords/StopWords_Geographic.txt",header=None)
stop_words7=pd.read_csv("/content/drive/MyDrive/Internship assignment /20211030 Test Assignment/StopWords/StopWords_Names.txt",header=None)

#combine all the stop words
stop_words=pd.concat([stop_wards1[1:], stop_words3[1:], stop_words4[1:], stop_words5[1:], stop_words6[1:], stop_words7[1:], stop_words2['col1'],stop_words2['col2'] ], axis=0)


In [ ]:
# Positive words 
positive_words=pd.read_csv("/content/drive/MyDrive/Internship assignment /20211030 Test Assignment/MasterDictionary/positive-words.txt")
positive_words=list(positive_words['a+'])
# Negative words
negative_words=pd.read_csv("/content/drive/MyDrive/Internship assignment /20211030 Test Assignment/MasterDictionary/negative-words.txt", encoding='latin-1')
negative_words=list(negative_words['2-faced'])

In [ ]:
#Data preprocessing for sentiment analysis

def data_preprocessing(data):
  ''' This function will prepare the data for sentiment analysis '''
  # Remove punctuation
  data = re.sub(r'[^\w\s]', '', data)
  
  # Remove numbers
  data = re.sub(r'[0,1,2,3,4,5,6,7,8,9]', '', data)
  
  # Remove double spaces
  data = data.replace("  ",' ')

  # Remove stop words
  filtered_words=[]

  for word in data.split():
    if word not in stop_words:
      filtered_words.append(word)

  data = ' '.join(filtered_words)

  # Making all the chars in lower case
  data = data.lower()

  return data


In [ ]:
# Sentiment Analysis

def sentiment_analysis(data):
  ''' This function will perform sentiment analysis and it will return the positive, negative, polarity and subjective score '''
  #prepare data for sentiment analysis
  data = data_preprocessing(data)

  # Tokenizing Data using nltks word_tokenize
  word_list = word_tokenize(data)
  
  # Calculate positive and negative score
  positive=0                    
  negative=0

  for word in word_list:          
    if word in positive_words:   #check point for positive word
      positive+=1
    elif word in negative_words: #check point for negative word
      negative-=1
    else:                       
      pass 
  positive, negative = positive, -(negative)                      
  
  # Calculate Polarity and Subjective score
  polarity = (positive - negative) / ((positive + negative) + 0.000001)
  subjective = (positive + negative)/ (len(word_list) + 0.000001)

  return positive, negative, round(polarity, 2), round(subjective, 2)

In [ ]:
#Applying sentiment analysis function
df[['Positive_score', 'Negative_score', 'Polarity_score', 'Subjectivity_score']] = df.apply(lambda x: sentiment_analysis(x['content']), axis=1, result_type = 'expand' )

In [ ]:
df.head(3)

,URL_ID,URL,content,Positive_score,Negative_score,Polarity_score,Subjectivity_score
0,37,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 millio...,77.0,34.0,0.39,0.06
1,38,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying ...",71.0,37.0,0.31,0.08
2,39,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the emp...,74.0,36.0,0.35,0.06


# Analysis of Readability

Average Sentence Length, Percentage of Complex words, Fog Index, Average Number of Words Per Sentenc, Complex Word Count

In [ ]:
def analysis_of_redability(data):
  ''' This function will parameters of readability such as avg_sent_len, complex_word_per, fog_index, avg_number_of_words_per_sentence, complex_word_count '''

  #Avg sentence length 
  total_sentence = len(sent_tokenize(data)) # tokanize the data based on snetence using nltk
  data = re.sub(r'[^\w\s]', '', data)  #remove punctuation
  total_words = len(word_tokenize(data)) # tokanize the data based on words using nltk
  avg_sent_len = round((total_words / total_sentence), 2)

  # Percentage of Complex Words - Thew word that contains more than two vowel sound is a complex word.
  count=0
  for word in data.split():
    if textstatistics().syllable_count(word) > 2: # textstatistics().syllable_count(word) it will check the number of vowel sounds in a word 
      count+=1

  complex_word_per = round((count / total_words*100) , 2)
  
  #Fog index
  fog_index = 0.4*(avg_sent_len + complex_word_per)

  ##Average number of words per sentence and average sentence length is a same 
  avg_number_of_words_per_sentence=avg_sent_len

  # Complex Word Count
  complex_word_count = count

  return avg_sent_len, complex_word_per, fog_index, avg_number_of_words_per_sentence, complex_word_count

In [ ]:
df[['Avg_ent_len', 'Complex_word_percentage', 'Fog_index','Avg_number_of_words_per_sentence' ,'Complex_word_count']] = df.apply(lambda x: analysis_of_redability(x['content']), axis = 1, result_type = 'expand')

In [ ]:
df.head(3)

,URL_ID,URL,content,Positive_score,Negative_score,Polarity_score,Subjectivity_score,Avg_ent_len,Complex_word_percentage,Fog_index,Avg_number_of_words_per_sentence,Complex_word_count
0,37,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 millio...,77.0,34.0,0.39,0.06,23.26,20.83,17.636,23.26,373.0
1,38,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying ...",71.0,37.0,0.31,0.08,17.88,13.29,12.468,17.88,190.0
2,39,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the emp...,74.0,36.0,0.35,0.06,20.06,20.00,16.024,20.06,341.0


#Word Count

In [ ]:
def word_count(data):
  ''' This function will return the number of words excluding stop words and puctuation'''
  
  # Word count
  data = re.sub(r'[^\w\s]', '', data)  #remove punctuation
  stop_words=stopwords.words('english') #stop words from nltk 

  filtered_words=[]
  for word in data.split():
    if word not in stop_words:
      filtered_words.append(word) 

  return len(filtered_words)

# Apply word count function  
df['Word_count'] = df['content'].apply(word_count)

#Syllable Count Per Word

In [ ]:
# Function to count number of vowel
def count_vowel(word):
  ''' This function will calculate number of vowels in a word'''
  count=0
  for i in word:
    if i in ('a', 'e', 'i', 'o', 'u'):
      count+=1
  return count

# Function to count syllable oer word
def syllable_per_word(data):
  ''' this function will return the average syllabuls per word'''
  data = data.lower()
  syllble_per_word=[]
  for word in data.split():
    if word.endswith('es') | word.endswith('ed'):  #Excluding words ends with 'es' & 'ed'
      pass
    else:
      syllble_per_word.append(count_vowel(word))   #Using count_vowel function
  return syllble_per_word

In [ ]:
df['Syllable_count_per_word'] = df['content'].apply(syllable_per_word)

#Personal Pronouns


In [ ]:
#Using regex fuction to count personal pronouns
pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
#Total personal pronouns in each input data
df['Personal_pronoun']=df['content'].apply(lambda x: len(pronounRegex.findall(x))) # pronounRegex.findall(x) gives all the personal pronouns present in a sentence

#Average Word Length

In [ ]:
def avg_word_length(data):
  ''' This function will reaturn average word length'''
  char=0
  for word in data.split():
    char+=len(word)
  return round(char/len(data.split()), )

df['Avg_word_length'] = df['content'].apply(avg_word_length)

In [ ]:
final_df = df.drop('content', axis=1)

In [ ]:
final_df.head()

,URL_ID,URL,Positive_score,Negative_score,Polarity_score,Subjectivity_score,Avg_ent_len,Complex_word_percentage,Fog_index,Avg_number_of_words_per_sentence,Complex_word_count,Word_count,Syllable_count_per_word,Personal_pronoun,Avg_word_length
0,37,https://insights.blackcoffer.com/ai-in-healthc...,77.0,34.0,0.39,0.06,23.26,20.83,17.636,23.26,373.0,1182,"[5, 1, 2, 1, 2, 0, 3, 3, 1, 1, 1, 1, 3, 1, 1, ...",1,6
1,38,https://insights.blackcoffer.com/what-if-the-c...,71.0,37.0,0.31,0.08,17.88,13.29,12.468,17.88,190.0,801,"[2, 1, 1, 5, 1, 2, 2, 1, 4, 1, 3, 3, 1, 1, 3, ...",8,5
2,39,https://insights.blackcoffer.com/what-jobs-wil...,74.0,36.0,0.35,0.06,20.06,20.00,16.024,20.06,341.0,1048,"[5, 2, 1, 2, 3, 1, 1, 3, 2, 4, 1, 2, 3, 3, 1, ...",3,6
3,40,https://insights.blackcoffer.com/will-machine-...,85.0,28.0,0.50,0.07,17.37,14.06,12.572,17.37,232.0,980,"[2, 1, 2, 2, 2, 1, 5, 5, 0, 1, 1, 1, 1, 5, 5, ...",19,5
4,41,https://insights.blackcoffer.com/will-ai-repla...,68.0,28.0,0.42,0.06,22.23,15.72,15.180,22.23,276.0,1037,"[3, 5, 1, 1, 1, 4, 1, 3, 1, 2, 1, 2, 1, 2, 1, ...",16,5


In [ ]:
# Create an output file
writer = pd.ExcelWriter('Output Data Structure.xlsx')
final_df.to_excel(writer, 'output')
writer.save()